In [27]:
import json
from collections import namedtuple
import re

import pandas as pd

Plan = namedtuple("Plan", "context actions")

goldpath_file = "../data/goldsequences-0-1-2-3-4-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21-22-23-24-25-26-27-28-29.json"
#goldpath_file = "../data/goldsequences-0.json"

with open(goldpath_file) as file:
    data = json.load(file)


In [2]:
len(data['0']['goldActionSequences'])

30

open/close OBJ - open/close a container
de/activate OBJ - activate/deactivate a device
connect OBJ to OBJ - connect electrical components
disconnect OBJ - disconnect electrical components
use OBJ [on OBJ] - use a device/item
look around describe - the current room
look at OBJ - describe an object in detail
look in OBJ - describe a container’s contents
read OBJ - read a note or book
move OBJ to OBJ - move an object to a container
pick up OBJ - move an object to the inventory
put down OBJ - drop an inventory item
pour OBJ into OBJ - pour a liquid into a container
dunk OBJ into OBJ - dunk a container into a liquid
mix OBJ - chemically mix a container
go to LOC - move to a new location
eat OBJ - eat a food
flush OBJ - flush a toilet
focus on OBJ - signal intent on a task object

In [4]:
def parse(observation: str, inventory: str = "") -> list[str]:
    try:
        x = re.search(r"([\S\s]*?)(?:In it, you see:)([\S\s]*?)(?:You also see:)([\S\s]*)", observation)
        if x == None:
            x = re.search(r"([\S\s]*?)(?:Here you see:)([\S\s]*?)(?:You also see:)([\S\s]*)", observation)
        groups = x.groups()

        location = groups[0]
        objects = groups[1]
        doors = groups[2]

        loc_split = [location.strip()]
        obs_split = [obs.strip() for obs in objects.split('\n') if len(obs.strip()) > 0]
        obs_split = [f"You see {obs}" for obs in obs_split]
        doors_split = [door.strip() for door in doors.split('\n') if len(door.strip()) > 0]
        inventory_items = inventory.replace('\n', ' ').replace('\t', '')
        return loc_split + obs_split + doors_split + [inventory_items]
    except Exception:
        print(observation)

In [ ]:
def get_plan_context(action: str) -> list[str]:


In [7]:
#action = "open door to bathroom"
action = "move metal pot to sink"

matcher = re.search(r"(?:move)([\S\s]*?)to([\S\s]*)", action)
if matcher:
    print([g.strip() for g in matcher.groups()])

['metal pot', 'sink']


In [11]:
variation = 3
all_plans = {}
for variation in range(0, len(data['0']['goldActionSequences'])):
    trajectory = data['0']['goldActionSequences'][variation]
    task = trajectory['taskDescription']
    goal = task.split('.')[0]
    if goal not in all_plans:
        all_plans[goal] = []

    plan = []

    previous_state = parse(trajectory['path'][variation]['freelook'])
    for step in trajectory['path'][1:]:
        current_state = parse(step['freelook'])
        action = step['action']
        preconditions = list(set(previous_state) - set(current_state))
        #plan.append((action, step['observation']))
        if action != 'look around':
            plan.append(action)
        previous_state = current_state
        if step['isCompleted'] == 'true':
            break

    all_plans[goal].append(plan)

print("ok")


ok


In [13]:
all_plans.keys()

dict_keys(['Your task is to boil water', 'Your task is to boil ice', 'Your task is to boil orange juice', 'Your task is to boil apple juice', 'Your task is to boil chocolate', 'Your task is to boil marshmallow', 'Your task is to boil ice cream', 'Your task is to boil soap', 'Your task is to boil rubber', 'Your task is to boil lead', 'Your task is to boil tin', 'Your task is to boil mercury', 'Your task is to boil gallium', 'Your task is to boil caesium'])

In [22]:
def common_member(a, b):
    a_set = set(a)
    b_set = set(b)

    # check length
    if len(a_set.intersection(b_set)) > 0:
        return(a_set.intersection(b_set))
    else:
        return("no common elements")

plans = all_plans['Your task is to boil water']
common_actions = []

plan_a = plans[0]

for plan in plans[1:]:
    common_actions = common_member(plan_a, plan)
    plan_a = common_actions

common_actions

{'activate sink',
 'activate stove',
 'deactivate sink',
 'examine substance in metal pot',
 'focus on substance in metal pot',
 'go to kitchen',
 'move metal pot to sink',
 'move metal pot to stove',
 'open cupboard',
 'open door to kitchen',
 'pick up metal pot',
 'pick up thermometer',
 'pour metal pot into metal pot',
 'use thermometer in inventory on substance in metal pot'}

In [28]:
rows = []
for i, plan in enumerate(plans):
    for j, action in enumerate(plan):
        flag = True if action in common_actions else False
        rows.append({
            'action': action,
            'landmark': flag,
            'plan': i,
            'order': j
        })

df = pd.DataFrame(rows)

In [33]:
df[df['plan'] == 0].sort_values("order")

,action,landmark,plan,order
0,open door to kitchen,True,0,0
1,go to kitchen,True,0,1
2,pick up thermometer,True,0,2
3,open cupboard,True,0,3
4,pick up metal pot,True,0,4
5,move metal pot to sink,True,0,5
6,activate sink,True,0,6
7,deactivate sink,True,0,7
8,pick up metal pot,True,0,8
9,focus on substance in metal pot,True,0,9


In [34]:
df[df['plan'] == 1].sort_values("order") # fazer planos NA MÃO

,action,landmark,plan,order
34,open door to hallway,False,1,0
35,go to hallway,False,1,1
36,open door to kitchen,True,1,2
37,go to kitchen,True,1,3
38,pick up thermometer,True,1,4
39,open cupboard,True,1,5
40,pick up metal pot,True,1,6
41,move metal pot to sink,True,1,7
42,activate sink,True,1,8
43,deactivate sink,True,1,9


In [ ]:
for i, row in df.iterrows():
    if not row['landmark'] or i == 0:
        #

In [88]:
obs = """
    This outside location is called the outside. Here see:
	the ground
	an axe
	a substance called wood
	a fountain (containing a substance called water)
	a fire pit (containing nothing)
	a substance called air
	the agent
You also see:
	A door to the kitchen (that is open)
	A door to the foundry (that is open)
	A door to the green house (that is open)
"""

x = re.search(r"([\S\s]*?)(?:Here you see:)([\S\s]*?)(?:You also see:)([\S\s]*)", obs)
x.groups()

AttributeError: 'NoneType' object has no attribute 'groups'

In [23]:
import difflib

def matches(list1, list2):
    while True:
        mbs = difflib.SequenceMatcher(None, list1, list2).get_matching_blocks()
        if len(mbs) == 1: break
        for i, j, n in mbs[::-1]:
            if n > 0: yield list1[i: i + n]
            del list1[i: i + n]
            del list2[j: j + n]

variations = [26, 27] # water

var_actions = []

for var in variations:
    actions = []
    for path in data['0']['goldActionSequences'][var]['path']:
        actions.append(path['action'])

    var_actions.append(actions)
print(var_actions[0])
print(var_actions[1])
list(matches(var_actions[0], var_actions[1]))

[['use thermometer in inventory on gaseous gallium', 'wait1'],
 ['pick up metal pot',
  'open door to outside',
  'go to outside',
  'open door to foundry',
  'go to foundry',
  'open blast furnace',
  'move metal pot to blast furnace',
  'activate blast furnace',
  'examine liquid gallium',
  'use thermometer in inventory on liquid gallium',
  'examine liquid gallium',
  'use thermometer in inventory on liquid gallium',
  'examine liquid gallium',
  'use thermometer in inventory on liquid gallium',
  'examine liquid gallium',
  'use thermometer in inventory on liquid gallium',
  'examine liquid gallium',
  'use thermometer in inventory on liquid gallium',
  'examine liquid gallium',
  'use thermometer in inventory on liquid gallium',
  'examine liquid gallium',
  'use thermometer in inventory on liquid gallium',
  'examine liquid gallium',
  'use thermometer in inventory on liquid gallium',
  'examine liquid gallium',
  'use thermometer in inventory on liquid gallium',
  'examine liqu

In [26]:
data['0']['goldActionSequences'][27]['path']

['look around',
 'open door to hallway',
 'go to hallway',
 'open door to kitchen',
 'go to kitchen',
 'look around',
 'pick up thermometer',
 'open cupboard',
 'pick up metal pot',
 'look around',
 'open freezer',
 'open glass jar',
 'open oven',
 'open drawer in cupboard',
 'open fridge',
 'open drawer in counter',
 'open door to bathroom',
 'go to bathroom',
 'look around',
 'open door to kitchen',
 'go to kitchen',
 'look around',
 'open door to outside',
 'go to outside',
 'look around',
 'open door to foundry',
 'go to foundry',
 'look around',
 'open door to outside',
 'go to outside',
 'look around',
 'open door to green house',
 'go to green house',
 'look around',
 'open door to hallway',
 'go to hallway',
 'look around',
 'open door to workshop',
 'go to workshop',
 'look around',
 'pick up gallium',
 'open door to hallway',
 'go to hallway',
 'open door to kitchen',
 'go to kitchen',
 'focus on gallium',
 'move gallium to metal pot',
 'pick up metal pot',
 'move metal pot t

In [71]:
task_example = data['0']['goldActionSequences'][0] # variation zero
print(task_example['taskDescription'])

plan = task_example['path']


Your task is to boil water. For compounds without a boiling point, combusting the substance is also acceptable. First, focus on the substance. Then, take actions that will cause it to change its state of matter.


In [89]:
#print(task_example['taskDescription'].split('.')[0])
#print(plan[0]['freelook'])

goal = task_example['taskDescription'].split('.')[0]
print(f"----------- goal: {goal} -------------")
actions = []
for idx, trajectory in enumerate(plan):
    action = trajectory['action']
    actions.append(action)
    state = trajectory['freelook']
    new_obs = trajectory['observation']
    inventory = trajectory['inventory']
    #if action != 'look around':
    print("--------------------------")
    print(f"Turn {idx}")
    print(f"State: {state}")
    print(f"inventory: {inventory}")
    print(f"Action: {action}")
    print(f"New obs: {new_obs}")


----------- goal: Your task is to boil water -------------
--------------------------
Turn 0
State: This room is called the hallway. In it, you see: 
	a picture
	a substance called air
	the agent
You also see:
	A door to the green house (that is open)
	A door to the living room (that is open)
	A door to the art studio (that is open)
	A door to the kitchen (that is open)
	A door to the bedroom (that is open)
	A door to the workshop (that is open)

inventory: In your inventory, you see:
	an orange

Action: look around
New obs: This room is called the hallway. In it, you see: 
	a picture
	a substance called air
	the agent
You also see:
	A door to the green house (that is open)
	A door to the living room (that is open)
	A door to the art studio (that is open)
	A door to the kitchen (that is open)
	A door to the bedroom (that is open)
	A door to the workshop (that is open)
--------------------------
Turn 1
State: This room is called the hallway. In it, you see: 
	a picture
	a substance call

In [ ]:
"""
! if your task is to boil an water AND you are not in the kitchen
    then
! go to the kitchen (you should see a door to the kitchen)
    + open door to the kitchen
    + go to the kitchen
! pick up metal pot (you should are in the kitchen)
    + open cupboard
    + pick up metal pot
+ pick up thermometer
! fill metal pot with water (must have metal pot in your inventory and you should see a sink)
    + pour metal pot into sink
    + move metal pot to sink
    + activate sink
    + deactivate sink
    + pick up metal pot
! boil water (You should have metal pot in your inventory with water and a thermometer, and you should see a stove)
    + deactivate stove
    + move metal pot to stove
    + activate stove
    + examine substance in metal pot
    + use thermometer in inventory on substance in metal pot
    + examine substance in metal pot
    + use thermometer in inventory on substance in metal pot
    + examine substance in metal pot
    + use thermometer in inventory on substance in metal pot
    + examine substance in metal pot
    + use thermometer in inventory on substance in metal pot
    + examine substance in metal pot
    + use thermometer in inventory on substance in metal pot
"""

In [87]:
# NLI test
"""
This room is called the kitchen. In it, you see: a substance called soapa painting a counter. On the counter is: a bowl (containing a banana, a potato, a red apple, an orange), a drawer.a sink, which is turned off. In the sink is: nothing.a table. On the table is: a glass cup (containing nothing).a chair. On the chair is: nothing.a freezer. The freezer door is closed. a lightera stopwatch, which is deactivated. a fridge. The fridge door is closed. a substance called aira cupboard. The cupboard door is open. In the cupboard is: a tin cup (containing nothing), a ceramic cup (containing nothing), a drawer.a oven, which is turned off. The oven door is closed. You also see:A door to the outside (that is open)A door to the bathroom (that is open)A door to the hallway (that is open) <SEP> you should see a closed door to the kitchen
"""

['look around',
 'open door to kitchen',
 'go to kitchen',
 'look around',
 'pick up thermometer',
 'open cupboard',
 'pick up metal pot',
 'look around',
 'move metal pot to sink',
 'activate sink',
 'deactivate sink',
 'pick up metal pot',
 'focus on substance in metal pot',
 'pour metal pot into metal pot',
 'pick up metal pot',
 'move metal pot to stove',
 'activate stove',
 'examine substance in metal pot',
 'use thermometer in inventory on substance in metal pot',
 'examine substance in metal pot',
 'use thermometer in inventory on substance in metal pot',
 'examine substance in metal pot',
 'use thermometer in inventory on substance in metal pot',
 'examine substance in metal pot',
 'use thermometer in inventory on substance in metal pot',
 'examine substance in metal pot',
 'use thermometer in inventory on substance in metal pot',
 'examine substance in metal pot',
 'use thermometer in inventory on substance in metal pot',
 'examine substance in metal pot',
 'use thermometer in

In [85]:
your goal is to boil water and (plan context) you are in the kitchen and you see an thermometer then
pick up thermometer
open cupboard
pick up metal pot
move metal pot to sink

SyntaxError: invalid syntax (2811270517.py, line 1)

In [81]:
len([a for a in actions if a != 'look around'])

37

In [66]:
if you are in {location} and you see a {obj}
then

18

In [ ]:
# comparar freelook do estado atual com o anterior
# o que mudar deve continuar no plan context
# o que não mudar, da pra remover no plan context, pois a ação não alterou o estado
# o plan context terá apenas